# Introduction to Classical Artificial Intelligence 
# 2 - Markov Decision Processes

This notebook is a modified version of the mdp_apps notebook of the AIMA 4th edition repository (https://github.com/aimacode/aima-python).

This notebook focuses on path planning problems.

Original preamble:
In this notebook we will take a look at some indicative applications of markov decision processes. 
We will cover content from [`mdp.py`](https://github.com/aimacode/aima-python/blob/master/mdp.py), for **Chapter 17 Making Complex Decisions** of Stuart Russel's and Peter Norvig's book [*Artificial Intelligence: A Modern Approach*](http://aima.cs.berkeley.edu/).


In [19]:
from mdp4e import *
from notebook import psource, pseudocode

## GRID MDP
---
Markov Decision Processes can be used to find the best path through a maze. Let us consider this simple maze.
![title](images/maze.png)

This environment can be formulated as a GridMDP.
<br>
To make the grid matrix, we will consider the state-reward to be -0.1 for every state.
<br>
State (1, 1) will have a reward of -5 to signify that this state is to be prohibited.
<br>
State (9, 9) will have a reward of +5.
This will be the terminal state.
<br>
The matrix can be generated using the GridMDP editor or we can write it ourselves.

In [20]:
grid = [
    [None, None, None, None, None, None, None, None, None, None, None], 
    [None, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, None, +5.0, None], 
    [None, -0.1, None, None, None, None, None, None, None, -0.1, None], 
    [None, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, None], 
    [None, -0.1, None, None, None, None, None, None, None, None, None], 
    [None, -0.1, None, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, None], 
    [None, -0.1, None, None, None, None, None, -0.1, None, -0.1, None], 
    [None, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, None, -0.1, None], 
    [None, None, None, None, None, -0.1, None, -0.1, None, -0.1, None], 
    [None, -5.0, -0.1, -0.1, -0.1, -0.1, None, -0.1, None, -0.1, None], 
    [None, None, None, None, None, None, None, None, None, None, None]
]

We have only one terminal state, (9, 9):

In [21]:
terminals = [(9, 9)]

We define our maze environment below:

In [22]:
maze = GridMDP(grid, terminals)

Let's see how value iteration works on a simple case.

In [23]:
def value_iteration_verbose(mdp, epsilon=0.001):
    """Solving an MDP by value iteration. [Figure 17.4]"""
    verb = '0'
    U1 = {s: 0 for s in mdp.states}
    R, T, gamma = mdp.R, mdp.T, mdp.gamma
    while True:
        U = U1.copy()
        if verb == '0':
            for key in U:
                print(key, ' : ', U[key])
            verb = input("Press 0 for step-by-step execution and 1 for fast execution: ")
        delta = 0
        for s in mdp.states:
            U1[s] = R(s) + gamma * max([sum([p * U[s1] for (p, s1) in T(s, a)])
                                        for a in mdp.actions(s)])
            delta = max(delta, abs(U1[s] - U[s]))
        if delta < epsilon * (1 - gamma) / gamma:
            return U

To solve the maze, we can use the `best_policy` function along with `value_iteration`.

In [24]:
toy_grid = GridMDP([[-0.04, -0.04, -0.04, +1],
        [-0.04, None,  -0.04, -1],
        [-0.04, -0.04, -0.04, -0.04]],
        terminals=[(3, 2), (3, 1)])

In [25]:
value_iteration_verbose(toy_grid)

(0, 1)  :  0
(1, 2)  :  0
(2, 1)  :  0
(0, 0)  :  0
(3, 1)  :  0
(2, 0)  :  0
(3, 0)  :  0
(0, 2)  :  0
(2, 2)  :  0
(1, 0)  :  0
(3, 2)  :  0
(0, 1)  :  -0.04
(1, 2)  :  -0.04
(2, 1)  :  -0.04
(0, 0)  :  -0.04
(3, 1)  :  -1.0
(2, 0)  :  -0.04
(3, 0)  :  -0.04
(0, 2)  :  -0.04
(2, 2)  :  -0.04
(1, 0)  :  -0.04
(3, 2)  :  1.0


{(0, 1): 0.3984432178350045,
 (1, 2): 0.649585681261095,
 (2, 1): 0.48644001739269643,
 (0, 0): 0.2962883154554812,
 (3, 1): -1.0,
 (2, 0): 0.3447542300124158,
 (3, 0): 0.12987274656746342,
 (0, 2): 0.5093943765842497,
 (2, 2): 0.7953620878466678,
 (1, 0): 0.25386699846479516,
 (3, 2): 1.0}

In [26]:
pi = best_policy(maze, value_iteration(maze))

Let's print out the best policy:

In [27]:
from utils import print_table
print_table(maze.to_arrows(pi))

None   None   None   None   None   None   None   None   None   None   None
None   v      <      <      <      <      <      <      None   .      None
None   v      None   None   None   None   None   None   None   ^      None
None   >      >      >      >      >      >      >      >      ^      None
None   ^      None   None   None   None   None   None   None   None   None
None   ^      None   >      >      >      >      v      <      <      None
None   ^      None   None   None   None   None   v      None   ^      None
None   ^      <      <      <      <      <      <      None   ^      None
None   None   None   None   None   ^      None   ^      None   ^      None
None   >      >      >      >      ^      None   ^      None   ^      None
None   None   None   None   None   None   None   None   None   None   None


As you can infer, we can find the path to the terminal state starting from any given state using this policy.
All maze problems can be solved by formulating it as a MDP.

In [28]:
grid2 = [
    [None, None, None, None, None, None, None, None, None, None, None], 
    [None, +5.0, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, +5.0, None], 
    [None, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, None, None, -0.1, None], 
    [None, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, None], 
    [None, -0.1, None, None, None, None, None, None, None, -0.1, None], 
    [None, -0.1, None, -0.1, -0.1, -0.1, -0.1, -0.1, None, -0.1, None], 
    [None, -0.1, None, -0.1, -0.1, -0.1, -0.1, -0.1, None, -0.1, None], 
    [None, -0.1, None, -0.1, -0.1, -0.1, -0.1, -0.1, None, -0.1, None], 
    [None, -0.1, None, -0.1, -0.1, -0.1, -0.1, -0.1, None, -0.1, None], 
    [None, -5.0, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, None], 
    [None, None, None, None, None, None, None, None, None, None, None]
]

In [29]:
maze2 = GridMDP(grid2, terminals)

In [30]:
pi2 = best_policy(maze2, value_iteration(maze2))

In [31]:
from utils import print_table
print_table(maze.to_arrows(pi2))

None   None   None   None   None   None   None   None   None   None   None
None   ^      <      <      <      <      <      <      <      .      None
None   ^      <      <      <      <      <      None   None   v      None
None   ^      <      <      <      <      <      <      <      <      None
None   ^      None   None   None   None   None   None   None   ^      None
None   ^      None   v      v      <      <      v      None   ^      None
None   ^      None   v      v      v      v      v      None   ^      None
None   ^      None   v      v      v      v      v      None   ^      None
None   ^      None   v      <      <      <      v      None   ^      None
None   ^      <      <      <      <      <      >      >      ^      None
None   None   None   None   None   None   None   None   None   None   None


<h1>Final Test</h1>

<h2>Question 2</h2>

In [35]:
#Define the grid for the path planning problem
test_grid = [
    [0,0,0,None,None,0,0,0,None],
    [0,0,0,0,0,0,None,0,0],
    [0,0,0,None,None,0,0,0,0],
    [None,None,0,None,0,0,0,0,None],
    [0,None,0,None,0,None,0,0,None],
    [0,0,0,None,0,None,None,0,0],
    [None,0,0,0,0,0,0,None,0],
    [None,None,None,0,0,None,0,0,0],
    [None,0,0,0,0,0,0,0,1],
    ]

#Define the terminals
test_terminal = [(8,8)]

#Define maze environmentfrom utils import print_table
test_maze = GridMDP(test_grid, test_terminal, gamma=0.01)

In [40]:
#find the best policy
test_pi = best_policy(test_maze, value_iteration(test_maze))


#print the best policy
from utils import print_table
print_table(test_maze.to_arrows(test_pi))

None   None   None   None   None   None   None   None   None
None   None   None   None   None   None   None   None   None
None   None   None   None   None   None   None   None   None
None   None   None   None   None   None   None   None   None
None   None   None   None   None   None   None   None   None
None   None   None   None   None   None   None   None   None
None   None   None   None   None   None   None   None   None
None   None   None   None   None   None   None   None   None
None   None   None   None   None   None   None   None   >   


<h2>Try out grid<h2>

In [43]:
#Define the grid for the path planning problem
try_grid = [
    [-1,-1,-1,None,None,-1,-1,-1,None],
    [-1,-1,-1,-1,-1,-1,None,-1,-1],
    [-1,-1,-1,None,None,-1,-1,-1,-1],
    [None,None,-1,None,-1,-1,-1,-1,None],
    [-1,None,-1,None,-1,None,-1,-1,None],
    [-1,-1,-1,None,-1,None,None,-1,-1],
    [None,-1,-1,-1,-1,-1,-1,None,-1],
    [None,None,None,-1,-1,None,-1,-1,-1],
    [None,-1,-1,-1,-1,-1,-1,-1,1],
    ]

#Define the terminals
try_terminal = [(8,8)]

#Define maze environmentfrom utils import print_table
try_maze = GridMDP(try_grid, try_terminal, gamma=0.01)

In [44]:
#find the best policy
try_pi = best_policy(try_maze, value_iteration(try_maze))


#print the best policy
from utils import print_table
print_table(try_maze.to_arrows(try_pi))

>      >      >      None   None   >      >      >      None
>      >      >      >      >      >      None   >      >   
>      >      >      None   None   >      >      >      >   
None   None   >      None   >      >      >      >      None
>      None   >      None   >      None   >      >      None
>      >      >      None   >      None   None   >      >   
None   >      >      >      >      >      >      None   >   
None   None   None   >      >      None   >      >      v   
None   >      >      >      >      >      >      >      >   
